In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import scipy
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os
import creep_event_picker as cep
import obspy
from matplotlib.dates import num2date
import matplotlib.dates as mdates
import h5py
import re
%matplotlib qt


In [8]:
def check_dir(path):
    isExist = os.path.exists(path)
    if not isExist:
        # Create a new directory because it does not exist 
        os.makedirs(path, exist_ok=True)

Import creepmeter database

In [9]:
Creepmeter_dataframe = pd.read_csv('../../Data/DATA_tidied/creepmeter_metadata_post_standardisation_sac_codes_updated.csv',index_col=0)
Creepmeter_dataframe

,Network,Creepmeter_abbrv,File_code,Start Time,End Time,"Sampling rate, Hz","Sampling rate, mins",Latitude,Longitude,Length,Depth,Obliquity,Creepmeter_full_name
0,EAF,BAL1,BAL1_1T,2023-03-12T10:47:55.000000Z,2023-09-05T13:14:55.000000Z,0.016667,1T,37.9903,38.1990,5 m,NaN,30.0,Balikburnu
1,PARK,C461,C461_10T,2004-09-29T02:20:00.000000Z,2022-06-20T12:40:00.000000Z,0.001667,10T,35.7240,-120.2820,NaN,NaN,30.0,Highway 46
2,PARK,C462,C462_1T,2021-08-19T19:10:11.000000Z,2024-03-22T01:05:11.000000Z,0.016667,1T,35.7240,-120.2820,NaN,NaN,30.0,Highway 46
3,UTA,CAN1,CAN1_30T,2012-06-12T15:33:00.000000Z,2017-06-15T08:33:00.000000Z,0.000556,30T,38.0198,-110.0374,7.5 m,NaN,45.0,Canyonlands
4,NAF,CER1,CER1_30S,2019-10-09T06:46:00.000000Z,2024-05-23T10:25:30.000000Z,0.033333,30S,40.8947,32.7773,20 m,NaN,29.0,Cerkes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,HOL,XSJ3,XSJ3_10T,2007-02-15T20:19:59.000000Z,2016-12-08T18:09:59.000000Z,0.001667,10T,36.8360,-121.5210,20 m,NaN,30.0,San Juan Bautista #3
125,PARK,XTA1,XTA1_10T,1985-12-18T00:00:00.000000Z,2024-10-09T18:50:00.000000Z,0.001667,10T,35.8900,-120.4270,30 m,1 m,30.0,Taylor Ranch
126,PARK,XVA1,XVA1_10T,1987-05-29T00:00:00.000000Z,2017-10-23T22:10:00.000000Z,0.001667,10T,35.9220,-120.4620,30 m,1 m,30.0,Varian
127,EAF,YZE1,YZE1_1T,2023-09-07T10:36:07.000000Z,2024-05-26T12:59:07.000000Z,0.016667,1T,38.1791,38.7526,20 m,NaN,30.0,Yazica East


In [10]:
frequency_dict = {
        1/60: '1S',    # 1 second
        1/6: '10S',  # 10 seconds
        1/2: '30S',  # 30 seconds
        1: '1T',  # 1 minute
        2: '2T',  # 2 minutes
        5: '5T',  # 5 minutes
        10: '10T',# 10 minutes
        15: '15T', # 15 minutes
        30: '30T',# 30 minutes
        60: '60T'    # 60 minutes
    }

In [11]:
np.where(Creepmeter_dataframe['Creepmeter_abbrv']=='NS45')

(array([59, 60, 61, 62]),)

In [12]:
plt.close('all')
for i in range(59,62):#len(Creepmeter_dataframe)):
    df_starts = pd.DataFrame(columns=['ST'])
    path = "../../Data/DATA_tidied/SAC/{q}.SAC".format(q =Creepmeter_dataframe.File_code.iloc[i])
    path_save = "../../Data/DATA_tidied/Picks/"
    check_dir(path_save)
    print(path)


    with h5py.File('../../Data/DATA_tidied/HDF5/{k}.h5'.format(k=Creepmeter_dataframe['Creepmeter_abbrv'].iloc[i]), 'r') as f:
        # Loop through each key in the file
        keys = list(f.keys())
        f.attrs
        for key in f.keys():
            starts = []
            ends = []
            event_slip = []  
            event_duration = []
            #event_code = []
            if key not in ['Temperature','Temperature_1T','Temperature_5T','Daily_measurements','Manual_measurements','Orthogonal']:
                print("Key:", key)
                print('extracting data')
                data = f[key]  # Access the dataset
                data_keys = list(data.keys())
                print(data_keys)
                slip = data[data_keys[0]][:]
                time =  data[data_keys[1]][:]
                decoded_time = [byte_str.decode('utf-8') for byte_str in time]
                decoded_time = pd.to_datetime(decoded_time)

                match = re.search(r'_(\d*\.?\d+)mins$', key)
                if match:
                    freq_value = float(match.group(1))
                    new_suffix = frequency_dict.get(freq_value)
                    if new_suffix:
                        new_key = re.sub(r'_\d*\.?\d+mins$', f'_{new_suffix}', key)
                    else:
                        new_key = key  # or raise an error / warning
                else:
                    new_key = key  # unchanged if pattern not matched

                print(new_key)



               
                tm_int = decoded_time
                creep_int = slip


                fig = plt.figure()
                ax = plt.subplot(1,1,1)
                plt.plot(tm_int,creep_int,marker='o', color='b', linestyle='-', linewidth=2, markersize=6)
                try:
                    df_picks = pd.read_csv("../../Data/DATA_tidied/Picks/{q}_picks.csv".format(q =new_key),index_col=0)
                except FileNotFoundError:
                    df_picks = pd.DataFrame({'ST':[],'ET':[]})
                
                for j in range(len(df_picks)):
                    try:
                        boolarr = np.logical_and(np.array(tm_int)>=pd.to_datetime(df_picks.ST_new.iloc[j]).replace(tzinfo=None),\
                                                        np.array(tm_int)<=pd.to_datetime(df_picks.ET_new.iloc[j]).replace(tzinfo=None))
                    except AttributeError:
                        boolarr = np.logical_and(np.array(tm_int)>=pd.to_datetime(df_picks.ST.iloc[j]).replace(tzinfo=None),\
                                                        np.array(tm_int)<=pd.to_datetime(df_picks.ET.iloc[j]).replace(tzinfo=None))
                    plt.plot(tm_int[boolarr],creep_int[boolarr])

                
                scale = 1.1
                zp = cep.ZoomPan()
                figZoom = zp.zoom_factory(ax, base_scale = scale)
                figPan = zp.pan_factory(ax)
                s = 1
                fig.set_size_inches(11,11)
                fig.tight_layout()
                plt.show()
                pts = plt.ginput(200,timeout = -1)
                plt.close('all')
                for sublist in pts:
                    df = pd.DataFrame({'ST':[sublist[0]],'SD':[sublist[1]]})
                    df_starts = pd.concat([df_starts,df])
                

                plt.show()
                if len(df_starts) >0:
                    #df_picks.reset_index(inplace=True)     
                    for r in range(len(df_starts)):
                        time_convert = mdates.num2date(df_starts.ST.iloc[r])
                        s3 = pd.Series(time_convert).dt.round("1min")
                        df_starts.ST.iloc[r] = s3.iloc[0].replace(tzinfo=None)   
                    
                    df_starts['ST'] = df_starts['ST'].astype(str).str[0:19]
                    df_starts.ST = pd.to_datetime(df_starts.ST)
                    df_starts.sort_values(by='ST',inplace=True)
                    
                    plt.close('all')
                    print(df_starts)
                    ##### define end ####
                    
                    df_end=pd.DataFrame(columns=['ET'])
                    fig = plt.figure()
                    ax = plt.subplot(1,1,1)
                    plt.plot(tm_int,creep_int)
                    for j in range(len(df_picks)):
                        try:
                            boolarr = np.logical_and(np.array(tm_int)>=pd.to_datetime(df_picks.ST_new.iloc[j]).replace(tzinfo=None),\
                                                        np.array(tm_int)<=pd.to_datetime(df_picks.ET_new.iloc[j]).replace(tzinfo=None))
                        except AttributeError:
                            boolarr = np.logical_and(np.array(tm_int)>=pd.to_datetime(df_picks.ST.iloc[j]).replace(tzinfo=None),\
                                                        np.array(tm_int)<=pd.to_datetime(df_picks.ET.iloc[j]).replace(tzinfo=None))
                        event_creep = creep_int[boolarr]
                        event_time = tm_int[boolarr]
                        ax.plot(event_time,event_creep,color='orange')
                    plt.scatter(pd.to_datetime(df_starts.ST),df_starts.SD,c='red')
                    for j in range(len(df_starts)):
                        ax.text(pd.to_datetime(df_starts.ST.iloc[j]),df_starts.SD.iloc[j],'{k}/{n}'.format(k=df_starts.index[j]+1,n=len(df_starts)))
                    scale = 1.1
                    zp = cep.ZoomPan()
                    figZoom = zp.zoom_factory(ax, base_scale = scale)
                    figPan = zp.pan_factory(ax)
                    s = 1
                    fig.set_size_inches(11,11)
                    #fig.tight_layout()
                    plt.show()
                    for j in range(len(df_starts)):
                        print('{m}/{n}'.format(m=df_starts.index[j]+1,n=len(df_starts)))
                        pts = plt.ginput(1,timeout = -1)
                        
                        for sublist in pts:
                            df = pd.DataFrame({'ET':[sublist[0]]})
                            df_end = pd.concat([df_end,df]) 
                    plt.close()

                    for r in range(len(df_end)):
                        time_convert = mdates.num2date(float(df_end.ET.iloc[r]))
                        s3 = pd.Series(time_convert).dt.round("1min")
                        df_end.ET.iloc[r] = s3.iloc[0].replace(tzinfo=None)  
                    
                    #df_picks_end.ET= mdates.num2date(df_picks_end.ET)
                    df_end.reset_index(inplace=True)
                    df_end.drop(columns=['index'],inplace=True)
                    
                    df_starts.reset_index(inplace=True)
                    #df_starts['ET_new'] = df_end['ET']
                    df_starts['ET'] = df_end['ET']
                    #df_starts['Quality_check'] = ['CE']*len(df_starts)
                    df_starts.drop(['index','SD'],axis=1,inplace=True)
                    df_picks = pd.concat([df_picks,df_starts])
                    #df_picks['ST_new'] = df_picks['ST_new'].astype(str).str[0:19]
                    #df_picks['ET_new'] = df_picks['ET_new'].astype(str).str[0:19]
                    df_picks['ST'] = df_picks['ST'].astype(str).str[0:19]
                    df_picks['ET'] = df_picks['ET'].astype(str).str[0:19]
                    print(df_picks)
                    df_picks.to_csv("../../Data/DATA_tidied/Picks/{q}_picks.csv".format(q =Creepmeter_dataframe.File_code.iloc[i]))

../../Data/DATA_tidied/SAC/NS45_1T.SAC
Key: NS45_1.0mins
extracting data
['Slip_NS45_1.0mins', 'Time_NS45_1.0mins']
NS45_1T
Key: NS45_10.0mins
extracting data
['Slip_NS45_10.0mins', 'Time_NS45_10.0mins']
NS45_10T


/var/folders/56/47k8q3q1449cq8kwzkz5r8ch0000gr/T/ipykernel_16001/3077966569.py:81: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_starts = pd.concat([df_starts,df])
/var/folders/56/47k8q3q1449cq8kwzkz5r8ch0000gr/T/ipykernel_16001/3077966569.py:90: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2017-11-30 00:00:00' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_starts.ST.iloc[r] = s3.iloc[0].replace(tzinfo=None)


          ST         SD
0 2017-11-30  16.806148
1/1


/var/folders/56/47k8q3q1449cq8kwzkz5r8ch0000gr/T/ipykernel_16001/3077966569.py:131: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_end = pd.concat([df_end,df])
/var/folders/56/47k8q3q1449cq8kwzkz5r8ch0000gr/T/ipykernel_16001/3077966569.py:137: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2018-01-08 06:01:00' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_end.ET.iloc[r] = s3.iloc[0].replace(tzinfo=None)


                    ST                   ET               ST_new  \
0  2018-01-09 12:20:00  2018-01-11 07:06:00  2018-01-09 10:00:00   
1  2018-01-21 17:39:00  2018-01-23 03:01:00  2018-01-21 12:40:00   
2  2018-01-31 15:00:00  2018-02-02 03:09:00  2018-01-31 11:00:00   
0  2017-11-30 00:00:00  2018-01-08 06:01:00                  NaN   

                ET_new Quality_check_new  
0  2018-01-12 23:00:00                CE  
1  2018-01-26 22:00:00                CE  
2  2018-02-06 00:30:00                CE  
0                  NaN               NaN  
Key: NS45_30.0mins
extracting data
['Slip_NS45_30.0mins', 'Time_NS45_30.0mins']
NS45_30T


TypeError: unsupported operand type(s) for *: 'Timestamp' and 'float'

In [16]:
df_picks

,ST,ET,ET_new,Quality_check
0,2023-03-07 10:27:00,2023-03-09 23:32:00,NaN,NaN
0,2023-03-18 17:52:00,2023-03-19 16:37:00,NaN,NaN
0,2023-05-07 08:22:00,2023-05-07 18:03:00,NaN,NaN
0,2023-09-04 09:10:00,2023-09-04 13:29:00,NaN,NaN
0,2023-10-25 06:00:00,2023-10-27 16:56:00,NaN,NaN
0,2023-11-06 13:58:00,2023-11-06 17:43:00,NaN,NaN
0,2023-11-19 03:54:00,2023-11-19 20:51:00,NaN,NaN
0,2023-12-09 12:15:00,2023-12-10 05:50:00,NaN,NaN
0,2023-12-10 08:38:00,2023-12-10 19:04:00,NaN,NaN
0,2023-03-27 14:59:00,NaN,2023-04-29 00:20:00,CE


In [17]:
Creepmeter_dataframe.File_code.iloc[i]

'KIR1_1T'

In [18]:
df_picks.to_csv("../../Data/DATA_tidied/Picks/{q}_picks.csv".format(q =Creepmeter_dataframe.File_code.iloc[i]))

In [ ]:
df_starts.reset_index(inplace=True)

In [ ]:
df_starts.to_csv("../../../Data/{p}/Picks/{q}_picks_starts.csv".format(p=Creepmeter_dataframe.Network.iloc[i],\
                                                                    q =Creepmeter_dataframe.File_code.iloc[i]))

In [6]:
df_picks

,ST,ET,Quality_check,ET_new,ST_new,Quality_check_new
0,2018-09-14 03:55:00,2018-09-14 06:21:00,NCE,2018-09-14 06:21:00,2018-09-14 03:55:00,NCE
1,2018-09-14 07:30:00,2018-09-14 15:00:00,CE,2018-09-15 11:14:00,2018-09-14 07:30:00,CE
2,2018-10-08 08:57:00,2018-10-08 09:51:00,NCE,2018-10-09 17:28:00,2018-10-08 08:57:00,CE
3,2018-10-08 10:19:00,2018-10-08 11:19:00,NCE,2018-10-08 11:19:00,2018-10-08 10:19:00,NCE
4,2018-10-08 13:39:00,2018-10-08 14:22:00,NCE,2018-10-08 14:22:00,2018-10-08 13:39:00,NCE
...,...,...,...,...,...,...
179,2022-12-02 14:18:00,2022-12-03 04:29:00,CE,2022-12-04 15:03:00,2022-12-02 14:18:00,CE
180,2022-12-22 03:29:00,2022-12-22 14:33:00,CE,2022-12-27 02:49:00,2022-12-22 03:29:00,CE
181,2023-02-08 23:58:00,2023-02-09 06:41:00,NCE,2023-02-14 15:44:00,2023-02-08 23:58:00,CE
182,2023-02-24 08:01:00,2023-02-24 10:45:00,NCE,2023-02-24 10:45:00,2023-02-24 08:01:00,CE


In [ ]:
i=0

In [ ]:
df_fixer = pd.read_csv("../../../Data/{p}/Picks/{q}_picks.csv".format(p=Creepmeter_dataframe.Network.iloc[i],\
                                                                    q =Creepmeter_dataframe.File_code.iloc[i]),index_col=0)

In [ ]:
df_fixer

In [ ]:
st_date = df_fixer.ST.astype(str).str[0:10]
st_time  = df_fixer.ST.astype(str).str[11:]
df_fixer.ST = st_date+ ' ' +st_time

In [ ]:
st_date = df_fixer.ET.astype(str).str[0:10]
st_time  = df_fixer.ET.astype(str).str[11:]
df_fixer.ET = st_date+ ' ' +st_time

In [ ]:
df_fixer

In [ ]:
df_fixer.to_csv("../../../Data/{p}/Picks/{q}_picks.csv".format(p=Creepmeter_dataframe.Network.iloc[i],\
                                                                    q =Creepmeter_dataframe.File_code.iloc[i]))

In [ ]:
for i in range(4,)